In [1]:
import numpy as np
import pandas as pd
from scipy import io
from PathOptimizationModel import moo_model, distance_soo_model
from PathSolution import PathSolution
from Time import get_real_paths
from Connectivity import enforce_hovering_connectivity

matlab_filepath = '/Users/kadircan/Documents/MATLAB/Thesis/ResultsTest'

model = moo_model

number_of_drones = 8
cell_comm_range = 4
min_visits = 1

# Get All Solutions and Objectives
scenario = f'g_8_a_50_n_{number_of_drones}_v_2.5_r_{cell_comm_range}_minv_{min_visits}_maxv_5_Nt_1_tarPos_12_ptdet_0.99_pfdet_0.01_detTh_0.9_maxIso_0'
sols = np.load(f'Results/X/{scenario}_SolutionObjects.npy', allow_pickle=True)
objs = np.load(f'Results/F/{scenario}_ObjectiveValues.npy', allow_pickle=True)
objs = pd.DataFrame(objs,columns=model['F'])

print(objs)

# Extract Sol with Best Connectivity
best_conn_idx = objs['Percentage Connectivity'].idxmin()
best_mean_disconn_idx = objs['Mean Disconnected Time'].idxmin()
best_max_disconn_idx = objs['Max Disconnected Time'].idxmin()
best_distance_idx = objs['Total Distance'].idxmin()

conn_sol:PathSolution = sols[best_conn_idx][0]
mean_disconn_sol:PathSolution = sols[best_mean_disconn_idx][0]
max_disconn_sol:PathSolution = sols[best_max_disconn_idx][0]
distance_sol:PathSolution = sols[best_distance_idx][0]

def check_real_paths(sol:PathSolution):
    x_matrix, y_matrix = get_real_paths(sol)
    print("X-Matrix:\n",pd.DataFrame(x_matrix).to_string(index=False))
    print("Y-Matrix:\n",pd.DataFrame(y_matrix).to_string(index=False))
    io.savemat(f"{matlab_filepath}/{scenario}-BestPercentage Connectivity-x_matrix.mat",{'array':x_matrix})
    io.savemat(f"{matlab_filepath}/{scenario}-BestPercentage Connectivity-y_matrix.mat",{'array':y_matrix})    

def check_connectivity(best_distance_sol:PathSolution):
    for time in range(sol.time_slots):
        print(f"step:{time}")
        drone_positions = sol.real_time_path_matrix[:,time]
        connectivity = sol.connectivity_matrix[time]
        connectivity_to_base = sol.connectivity_to_base_matrix[time]
        print(f"drone positions:\n{drone_positions}")
        print(f"connectivity matrix:\n{connectivity}")
        print(f"connectivity to base matrix:\n{connectivity_to_base}")

# print(f"Path Matrix:\n{sol.real_time_path_matrix}")

check_real_paths(conn_sol)
# hovering_connectivity_constraint(conn_sol)

'''real_time_path_matrix = sol.real_time_path_matrix
time_slots = sol.time_slots
print(time_slots)
print(real_time_path_matrix.shape[1])

print(sol.real_time_path_matrix)
print(sol.connectivity_matrix)
print(sol.connectivity_to_base_matrix)
'''

# best_conn_value = objs.loc[objs['Percentage Connectivity'].idxmin()]
# best_conn_value

/Users/kadircan/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/kadircan/Desktop/Portfolio/Projects/sar_with_hovering/Time.py:186: RuntimeWarning: invalid value encountered in divide
  drone_speeds = drone_dists / step_time


   Total Distance  Percentage Connectivity  Mean Disconnected Time  \
0     6027.133175                -0.990385                    0.25   
1     6004.845590                -0.980769                    0.25   
2     6027.133175                -0.990385                    0.25   
3     6033.164552                -1.000000                    0.00   

   Max Disconnected Time  
0                    2.0  
1                    2.0  
2                    2.0  
3                    0.0  
X-Matrix:
  0          1          2          3          4          5          6          7          8          9          10         11         12         13         14         15         16         17         18         19         20         21         22         23         24         25         26         27        28         29         30         31         32         33         34         35         36         37         38         39         40         41         42         43         44         45      

'real_time_path_matrix = sol.real_time_path_matrix\ntime_slots = sol.time_slots\nprint(time_slots)\nprint(real_time_path_matrix.shape[1])\n\nprint(sol.real_time_path_matrix)\nprint(sol.connectivity_matrix)\nprint(sol.connectivity_to_base_matrix)\n'

In [ ]:
# PATH ANIMATION TEST

# Path animation function

from PathInfo import *
from PathSolution import *

scenario_info = {
    'grid_size': 8,
    'cell_side_length': 50,
    'number_of_drones': 4,
    'max_drone_speed': 2.5, # m/s
    'comm_cell_range': 4,  # 2 cells
    'min_visits': 1,  # Minimum number of cell visits
    'max_visits':5, # Maximum number of cell visits
    'number_of_targets': 1,
    'target_positions':12,
    'true_detection_probability': 0.99,
    'false_detection_probability': 0.01,
    'detection_threshold': 0.9,
    'max_isolated_time': 0,
}